# 📘 代理架构 7：黑板系统

欢迎来到我们代理架构系列的第七个notebook。今天，我们探索**黑板系统**，一个强大且高度灵活的模式，用于协调多个专家代理。 这种架构基于一组人类专家围绕物理黑板协作解决复杂问题的理念建模。

与刚性的、预定义的代理交接序列不同，黑板系统具有一个中央共享数据存储（'黑板'），代理可以在其中读取问题的当前状态并写入他们的贡献。 一个动态的**控制器**观察黑板并根据推进解决方案所需决定下一步激活哪个专家代理。这允许机会主义和涌现的工作流程。

为了突出其独特优势，我们将其与我们之前构建的**顺序多代理系统**进行比较。我们将向两个系统提出一个复杂的金融查询，其中最优路径不是简单的→ B → C序列。我们将演示刚性顺序代理如何遵循次优路径，而黑板系统的动态控制器以更合理、数据驱动的顺序激活代理，从而产生更高效和连贯的分析。

### 定义
**黑板系统**是一种多代理架构，其中多个专家代理通过读取和写入称为'黑板'的共享中央数据存储库进行协作。 控制器或调度器根据黑板上解决方案的演变状态动态确定下一个应该行动的代理。

### 高级工作流程

1. **共享内存（黑板）：** 一个中央数据结构保存问题的当前状态，包括用户请求、中间发现和部分解决方案。
2. **专家代理：** 一组独立的代理，每个都具有特定的专业知识，持续监控黑板。
3. **控制器：** 一个中央'控制器'代理也监控黑板。它的工作是分析当前状态并决定哪个专家代理最适合做出下一个贡献。
4. **机会主义激活：** 控制器激活选定的代理。代理从黑板读取相关数据，执行其任务，并将其发现写回黑板。
5. **迭代：**过程重复，控制器以动态序列激活不同的代理，直到它确定黑板上的解决方案已完成。

### 何时使用/应用场景
* **复杂、结构不良的问题：** 适用于解决方案路径事先未知且需要涌现、机会主义策略的问题（例如，复杂诊断、科学发现）。
* **多模态系统：** 协调处理不同数据类型（文本、图像、代码）的代理的好方法，因为它们都可以将发现发布到共享黑板。
* **动态意义构建：** 需要从许多不同的异步来源综合信息的情况。

### 优点和缺点
* **优点：**
 * **灵活性和适应性：** 工作流程不是硬编码的；它根据问题涌现，使系统高度自适应。
 * **模块化：** 很容易添加或删除专家代理而无需重新架构整个系统。
* **缺点：**
 * **控制器复杂性：** 整个系统的智能严重依赖于控制器的复杂程度。天真的控制器可能导致低效或循环行为。
 * **调试挑战：** 工作流程的非线性、涌现性质有时可能使其比简单的顺序过程更难追踪和调试。

## 阶段0：基础与设置

我们将从标准设置过程开始：安装库并配置Nebius、LangSmith和Tavily的API密钥。

### 步骤0.1： 安装核心库

**我们将要做的：**
我们将为这个项目系列安装标准的库套件。

In [1]:
# !pip install -q -U langchain-nebius langchain langgraph rich python-dotenv langchain-tavily

### 步骤0.2： 导入库和设置密钥

**我们将要做的：**
我们将导入必要的模块并从`.env`文件加载我们的API密钥。

**需要执行的操作：** 在此目录中创建一个包含您的密钥的`.env`文件：
```
NEBIUS_API_KEY="your_nebius_api_key_here"
LANGCHAIN_API_KEY="your_langsmith_api_key_here"
TAVILY_API_KEY="your_tavily_api_key_here"
```

In [2]:
import os from typing import List, Annotated, TypedDict, Optional from dotenv import load_dotenv

# LangChain components from langchain_nebius import ChatNebius from langchain_tavily import TavilySearch from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage from pydantic import BaseModel, Field from langchain_core.prompts import ChatPromptTemplate

# LangGraph components from langgraph import StateGraph, END

# 用于美观打印 from rich.console import Console from rich.markdown import Markdown

# --- API KeyandTracing Setup ---
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Agentic Architecture - Blackboard (Nebius)" for key in["NEBIUS_API_KEY", "LANGCHAIN_API_KEY", "TAVILY_API_KEY"]:
 if not os.environ.get(key):
 print(f"{key} 未找到。请创建.env文件并设置密钥。")

print("环境变量已加载，追踪设置已完成。")

环境变量已加载，追踪设置已完成。


## 阶段1：基线 - 修正的顺序多代理系统

为了理解黑板的灵活性，我们首先需要一个正确运行的顺序系统。原始版本失败是因为专家没有使用前面步骤的输出。我们将通过确保每个代理从状态接收必要的上下文来纠正这一点。

### 步骤1.1：构建修正的顺序团队

**我们将要做的：**
我们将定义明确使用其前任输出的专家代理，然后将它们连接成固定的线性序列。

In [3]:
console = Console()
# Using a more capable model to handle complex instructions better
llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0)
search_tool = TavilySearch(max_results=2)

# State for sequential agent
class SequentialState(TypedDict):
 user_request: str
 news_report: Optional[str]
 technical_report: Optional[str]
 financial_report: Optional[str]
 final_report: Optional[str]

# --- CORRECTED SPECIALIST NODES FOR SEQUENTIAL AGENT ---
# Key change is that each agent now gets context from previous steps, not just original request.

def news_analyst_node_seq(state: SequentialState):
 console.print("--- (Sequential) 调用新闻分析师 ---")
 prompt = f"你的任务是作为专业新闻分析师。查找用户请求中主题的最新重大新闻并提供简洁摘要。\n\n用户请求: {state['user_request']}"
 agent = llm.bind_tools([search_tool])
 result = agent.invoke(prompt)
 return {"news_report": result.content}

def technical_analyst_node_seq(state: SequentialState):
 console.print("--- (Sequential) 调用技术分析师 ---")
 # This agent now uses news report as context.
 prompt = f"你的任务是作为专业技术分析师。基于以下新闻报告，对公司股票进行技术分析。\n\n新闻报告:\n{state['news_report']}"
 agent = llm.bind_tools([search_tool])
 result = agent.invoke(prompt)
 return {"technical_report": result.content}

def financial_analyst_node_seq(state: SequentialState):
 console.print("--- (Sequential) 调用财务分析师 ---")
 # This agent also uses news report as context.
 prompt = f"你的任务是作为专业财务分析师。基于以下新闻报告，分析公司最近的财务表现。\n\n新闻报告:\n{state['news_report']}"
 agent = llm.bind_tools([search_tool])
 result = agent.invoke(prompt)
 return {"financial_report": result.content}


def report_writer_node_seq(state: SequentialState):
 console.print("--- (Sequential) 调用报告撰写者 ---")
 prompt = f"""你是一名专业的报告撰写者。你的任务是将新闻、技术和财务分析师的信息综合成一份直接回答用户原始请求的连贯报告。

用户请求: {state['user_request']}

以下是要合并的报告：
---
新闻报告: {state['news_report']}
---
技术报告: {state['technical_report']}
---
财务报告: {state['financial_report']}
"""
 report = llm.invoke(prompt).content
 return {"final_report": report}

# Build sequential graph
seq_graph_builder = StateGraph(SequentialState)
seq_graph_builder.add_node("news", news_analyst_node_seq)
seq_graph_builder.add_node("tech", technical_analyst_node_seq)
seq_graph_builder.add_node("finance", financial_analyst_node_seq)
seq_graph_builder.add_node("writer", report_writer_node_seq)

# Rigid, hardcoded sequence
seq_graph_builder.set_entry_point("news")
seq_graph_builder.add_edge("news", "tech")
seq_graph_builder.add_edge("tech", "finance")
seq_graph_builder.add_edge("finance", "writer")
seq_graph_builder.add_edge("writer", END)

sequential_app = seq_graph_builder.compile()
print("修正的顺序多代理系统编译成功。")

修正的顺序多代理系统编译成功。


### 步骤1.2：在动态问题上测试顺序代理

现在顺序代理正确传递上下文，让我们观察其行为。它将产生更连贯的报告，但其*过程*仍将是低效的，并且无法遵循条件逻辑。

In [4]:
dynamic_query = "查找关于Nvidia的最新重大新闻。基于该新闻的情绪，进行技术分析（如果新闻是中性或积极的）或对其最近表现的财务分析（如果新闻是负面的）。"

console.print(f"[bold yellow]测试修正的顺序代理在动态查询上:[/bold yellow]\n'{dynamic_query}'\n")

# Run graph
final_seq_output = sequential_app.invoke({"user_request": dynamic_query})

console.print("\n--- [bold red]顺序代理的最终报告[/bold red] ---")
console.print(Markdown(final_seq_output['final_report']))

测试修正的顺序代理在动态查询上：
'查找关于Nvidia的最新重大新闻。基于该新闻的情绪，进行技术分析（如果新闻是中性或积极的）或对其最近表现的财务分析（如果新闻是负面的）。'


--- (Sequential) 调用新闻分析师 ---
--- (Sequential) 调用技术分析师 ---
--- (Sequential) 调用财务分析师 ---
--- (Sequential) 调用报告撰写者 ---



--- 顺序代理的最终报告 ---


### Nvidia最新新闻综合分析

**用户请求：** 查找关于Nvidia的最新重大新闻。 基于该新闻的情绪，进行技术分析（如果新闻是中性或积极的）或财务分析（如果新闻是负面的）。

**1. 新闻摘要：**
关于Nvidia的最新重大新闻围绕着他们下一代AI芯片架构的公告, codenamed "Rubin." 这一新闻在市场上产生了显著的积极情绪, 分析师强调了在AI硬件领域持续主导地位的潜力. announcement points 到roadmap extending beyond current "Blackwell" chips, 表明持续的创新和增长. 这一新闻是压倒性积极的.

**2. 技术分析：**
在积极新闻之后，Nvidia的股票（NVDA）显示出强劲的看涨指标. 股价已突破之前的阻力位 并且交易价格远高于其50日和200日移动平均线. 上涨日的交易量很高，确认了看涨趋势. 关键指标如相对强弱指数（RSI）处于超买区域, 表明可能出现短期回调, 但整体动量仍然强劲积极.

**3. 财务分析：**
Nvidia最近的财务表现异常强劲, 受其数据中心GPU需求激增的推动. 公司最新的季度收益报告显示收入和净利润数据大幅超出分析师预期. 毛利率已扩大, 公司对即将到来的季度提供了非常乐观的预测, 进一步增强了投资者信心.

**结论：**
突破性产品公告、积极的市场情绪、强劲的技术股票表现和创纪录的财务业绩的组合为Nvidia描绘了非常看涨的图景. 虽然用户的请求指定了基于情绪的'要么/要么'分析，但为了完整性，本报告包含了所有分析, 从多个角度确认了公司的稳健地位.

**修正后输出的讨论：**
代理现在产生完整、合理的报告。然而，执行跟踪`News → Technical → Financial`揭示了其根本缺陷。它执行了**技术和财务分析两者**，完全忽略了用户的条件请求（"要么...要么..."）。这是低效的，展示了我们旨在用黑板架构解决的刚性。

## 阶段2：高级方法 - 修正的黑板系统

现在，我们将构建黑板系统。修复原始循环行为的关键是为**控制器**制作一个更智能的提示，使其意识到自己作为有状态规划器的角色。

### 步骤2.1：定义黑板和修正的控制器

**我们将要做的：**
1. **黑板状态：** 定义`BlackboardState`作为共享内存。
2. **专家代理：** 定义专家节点。它们将类似于我们之前的代理。
3. **控制器（修正）：** 创建一个健壮的`controller_node`，其提示明确推理已完成的步骤和剩余目标。这是最关键的更改。

In [5]:
# Blackboard State holds all information
class BlackboardState(TypedDict):
 user_request: str
 # Central blackboard where agents post their findings as strings
 blackboard: List[str]
 # List of available agents for controller to choose fromavailable_agents: List[str]
 # Controller's next decision
 next_agent: Optional[str]

# Pydantic model for Controller's decision
# CORRECTION: Added list of available agents to field description to guide LLM's choice.
class ControllerDecision(BaseModel):
 next_agent: str = Field(description="要调用的下一个代理的名称。必须是['新闻分析师', '技术分析师', '财务分析师', '报告撰写者']之一或'FINISH'。")
 reasoning: str = Field(description="选择下一个代理的简要原因。")

# Reusable factory for creating specialist agents for blackboard
def create_blackboard_specialist(persona: str, agent_name: str):
 system_prompt = f"""你是一名专业的专家代理：{persona}.
你的任务是通过执行你的特定功能来为更大的目标做贡献。
阅读初始用户请求和当前黑板以获取上下文。
使用你的工具查找所需信息。
最后，将你简洁的markdown报告发布回黑板。你的报告应该用你的名字签名 '{agent_name}'。
"""
 prompt_template = ChatPromptTemplate.from_messages([
 ("system", system_prompt),
 ("human", "用户请求: {user_request}\n\n黑板（之前的报告）:\n{blackboard_str}")
 ])
 agent = prompt_template | llm.bind_tools([search_tool])

 def specialist_node(state: BlackboardState):
 console.print(f"--- (黑板) 代理 '{agent_name}' 正在工作... ---")
 blackboard_str = "\n---\n".join(state["blackboard"])
 result = agent.invoke({"user_request": state["user_request"], "blackboard_str": blackboard_str})
 报告= f"**报告来自{agent_name}:**\n{result.content}"
 # Append new report to list of blackboard entries
 return {"blackboard": state["blackboard"] + [report]}
 return specialist_node

# Create specialist agent nodes
news_analyst_bb = create_blackboard_specialist("News Analyst", "News Analyst")
technical_analyst_bb = create_blackboard_specialist("Technical Analyst", "Technical Analyst")
financial_analyst_bb = create_blackboard_specialist("Financial Analyst", "Financial Analyst")
report_writer_bb = create_blackboard_specialist("从黑板综合最终答案的报告撰写者", "Report Writer")

# --- THE CORRECTED, INTELLIGENT CONTROLLER NODE ---
# This is the most important fix. The prompt is now much more sophisticated.
def controller_node(state: BlackboardState):
 console.print("--- 控制器: 分析黑板中... ---")

 # Use a structured output LLM to make decision
 controller_llm = llm.with_structured_output(ControllerDecision)

 blackboard_content = "\n\n".join(state['blackboard'])
 agent_list = state['available_agents']

 # New prompt is state-aware and goal-oriented.
 prompt = f"""你是多代理系统的中央控制器。你的工作是分析共享黑板和原始用户请求，决定下一个应该运行哪个专家代理。

**原始用户请求：**
{state['user_request']}

**当前黑板内容：**
---
{blackboard_content if blackboard_content else "黑板当前为空。"}
---

**可用专家代理：**
{', '.join(agent_list)}

**你的任务：**
1. 仔细阅读用户请求和当前黑板内容。
2. 确定*下一个逻辑步骤*是什么以更接近完整答案。
3. 从可用代理列表中选择单个最佳代理来执行该步骤。
4. 如果用户请求已完全解决且最终报告已撰写，选择'FINISH'。在"Report Writer"提供最终综合答案之前不要完成。

以要求的格式提供你的决定。
"""
 decision_result = controller_llm.invoke(prompt)
 console.print(f"--- 控制器: 决定调用 '{decision_result.next_agent}'。原因：{decision_result.reasoning} ---")

 # dictionary returned here updates 'next_agent' keyingraph's state
 return {"next_agent": decision_result.next_agent}

print("黑板组件和修正的控制器节点已定义。")

黑板组件和修正的控制器节点已定义。


### 步骤2.2：构建黑板图

现在我们将组件连接成动态状态图。控制器充当中央路由器。任何专家运行后，控制总是返回到控制器来决定下一步。

In [6]:
bb_graph_builder = StateGraph(BlackboardState)

# Add all nodes to graph
bb_graph_builder.add_node("Controller", controller_node)
bb_graph_builder.add_node("新闻分析师", news_analyst_bb)
bb_graph_builder.add_node("技术分析师", technical_analyst_bb)
bb_graph_builder.add_node("财务分析师", financial_analyst_bb)
bb_graph_builder.add_node("报告撰写者", report_writer_bb)

bb_graph_builder.set_entry_point("Controller")

# This function defines dynamic routing logic based on Controller's decision
def route_to_agent(state: BlackboardState):
 returnstate["next_agent"]

# Conditional edges route from Controller to chosen specialist or to end
bb_graph_builder.add_conditional_edges(
 "Controller",
 route_to_agent,
 {
 "新闻分析师": "新闻分析师",
 "技术分析师": "技术分析师",
 "财务分析师": "财务分析师",
 "报告撰写者": "报告撰写者",
 "FINISH": END
 }
)

# After any specialist runs, control always returns to Controller for next decision
bb_graph_builder.add_edge("新闻分析师", "Controller")
bb_graph_builder.add_edge("技术分析师", "Controller")
bb_graph_builder.add_edge("财务分析师", "Controller")
bb_graph_builder.add_edge("报告撰写者", "Controller")

blackboard_app = bb_graph_builder.compile()
print("黑板系统编译成功。")

黑板系统编译成功。


## 阶段3：正面对比

让我们在相同的动态任务上运行我们新的黑板系统并观察其智能工作流程。

In [7]:
console.print(f"[bold green]测试黑板系统在相同的动态查询上:[/bold green]\n'{dynamic_query}'\n")

agent_list = ["新闻分析师", "技术分析师", "财务分析师", "报告撰写者"]
initial_bb_input = {"user_request": dynamic_query, "blackboard": [], "available_agents": agent_list}

# 我们使用stream来观察逐步过程
final_bb_output = None for chunk inblackboard_app.stream(initial_bb_input, {"recursion_limit": 10}):
 final_bb_output = chunk
 console.print("\n--- [bold purple]当前黑板状态[/bold purple] ---")
 # 美观打印黑板中的每个报告
 for i, report in enumerate(final_bb_output.get('blackboard', [])):
 console.print(f"--- 报告 {i+1} ---")
 console.print(Markdown(report))
 console.print("\n")

console.print("\n--- [bold green]黑板系统最终报告[/bold green] ---")
# 最终报告是撰写者发布到黑板的最后一项
final_report_content = final_bb_output['blackboard'][-1]
console.print(Markdown(final_report_content))

测试黑板系统在相同的动态查询上:
'查找关于Nvidia的最新重大新闻。基于该新闻的情绪，进行技术分析（如果新闻是中性或积极的）或对其最近表现的财务分析（如果新闻是负面的）。'


--- 控制器: 分析黑板中... ---
--- 控制器: 决定调用 '新闻分析师'。原因： 黑板为空，所以第一步是按照用户请求收集关于Nvidia的最新新闻。 ---



--- 当前黑板状态 ---



--- (黑板) 代理 '新闻分析师' 正在工作... ---
--- 控制器: 分析黑板中... ---
--- 控制器: 决定调用 '技术分析师'。原因： 黑板中的新闻报告是积极的，提到了新的芯片架构和市场主导地位。根据用户的请求，对于积极新闻，技术分析是正确的下一步。 ---



--- 当前黑板状态 ---


...

**技术分析师报告：**
基于关于Nvidia加速路线图的积极新闻, NVDA股票的技术分析显示强劲的看涨动量. 股票目前处于明显的上升趋势, 交易价格显著高于其50日和200日简单移动平均线（SMA）. 公告在价格图表上创造了看涨缺口, 该缺口已保持，表明强劲的买入压力. 相对强弱指数（RSI）很高, 表明股票处于超买区域, 但这对于处于强劲上升趋势的股票来说很常见. 关键支撑现在建立在最近价格缺口的顶部.

--- 报告 3 ---


**报告撰写者报告：**
### 基于最新新闻的Nvidia最终分析

**初始请求：** 查找关于Nvidia的最新重大新闻。 基于该新闻的情绪，进行技术分析（如果新闻是中性或积极的）或财务分析（如果新闻是负面的）。

**过程摘要：**
1. **新闻分析：** Nvidia的最新重大新闻是其下一代的公告 "Rubin" AI chip platform, 产生了高度积极的市场情绪.
2. **条件操作：** 根据用户的请求，积极的新闻情绪触发了技术分析.
3. **技术分析：** NVDA股票的技术前景强劲看涨. 股票处于明显的上升趋势，交易价格高于关键移动平均线, 最近的价格缺口确认了显著的买入压力.

**结论：** 遵循用户的指令，对Nvidia加速创新的近期积极新闻进行分析，导致了技术审查，确认了公司股票的强劲看涨动量。


--- 黑板系统的最终报告 ---


**报告撰写者报告：**
### 基于最新新闻的Nvidia最终分析

**初始请求：** 查找关于Nvidia的最新重大新闻。 基于该新闻的情绪，进行技术分析（如果新闻是中性或积极的）或财务分析（如果新闻是负面的）。

**过程摘要：**
1. **新闻分析：** Nvidia的最新重大新闻是其下一代的公告 "Rubin" AI chip platform, 产生了高度积极的市场情绪.
2. **条件操作：** 根据用户的请求，积极的新闻情绪触发了技术分析.
3. **技术分析：** NVDA股票的技术前景强劲看涨. 股票处于明显的上升趋势，交易价格高于关键移动平均线, 最近的价格缺口确认了显著的买入压力.

**结论：** 遵循用户的指令，对Nvidia加速创新的近期积极新闻进行分析，导致了技术审查，确认了公司股票的强劲看涨动量。

**修正后输出的讨论：**
成功！`GraphRecursionError`已消失。执行跟踪揭示了一个更智能的过程：

1. **控制器启动：** 控制器启动，看到空黑板，正确决定首先调用**新闻分析师**。
2. **新闻分析师运行：** 新闻分析师找到最新新闻并将其报告发布到黑板。
3. **控制器重新评估：** 控制返回到控制器。它读取新闻分析师的报告，理解情绪，并遵循用户的逻辑。它智能地决定调用适当的下一个分析师（**技术**或**财务**），完全跳过另一个。
4. **专家运行：** 选定的分析师执行其任务并将其报告添加到黑板。
5. **控制器完成：** 控制器看到所有必要的分析已完成，并调用**报告撰写者**来综合最终答案。
6. **最终调用：** 撰写者发布最终报告后，控制器看到这一点并决定**完成**。

这种动态、机会主义的工作流程是正常运行的黑板系统的标志。它完美地遵循了用户的复杂条件逻辑，节省了时间和资源。

## 阶段4：定量评估

为了正式化比较，我们将使用LLM作为评判者来评估两个系统在指令遵循和过程效率方面的表现。

In [8]:
class ProcessLogicEvaluation(BaseModel):
 """Schema for evaluating agent's logical process."""
 instruction_following_score: int = Field(description="1-10分评估代理遵循用户特定条件指令的程度。")
 process_efficiency_score: int = Field(description="1-10分评估代理是否采取了最直接的路径并避免了不必要的工作。")
 justification: str = Field(description="评分的简要理由，引用代理采取的具体步骤。")

# Use a strong model for judging
judge_llm = ChatNebius(model="mistralai/Mixtral-8x22B-Instruct-v0.1", temperature=0).with_structured_output(ProcessLogicEvaluation)

def evaluate_agent_logic(query: str, final_state: dict):
 # Reconstruct a simplified trace for the judge
 trace = ""
 agent_type = "Unknown"
 if 'blackboard' in final_state: # Blackboard agent
 agent_type = "Blackboard"
 trace = "\n---\n".join(final_state['blackboard'])
 else: # Sequential agent
 agent_type = "Sequential"
 trace = f"1. News Report Generated: {final_state.get('news_report')}\n---\n2. Technical Report Generated: {final_state.get('technical_report')}\n---\n3. Financial Report Generated: {final_state.get('financial_report')}"

 prompt = f"""你是AI代理流程的专业评判员。你的任务是基于其生成的内容跟踪评估代理的性能。

**User's Original Task:**
"{query}"

**Agent's Type:** {agent_type}
**Agent's Generated content Trace:**
```
{trace}
```

**评估 Criteria:**
1. **Instruction Following:** 代理是否遵守了用户任务中的条件逻辑？ （例如，"要么技术分析...要么财务分析"）. 高分意味着它完美遵循了逻辑。低分意味着它忽略了逻辑。
2. **过程 效率:** 代理是否避免了不必要的工作？ 高分意味着它只运行了必需的专家。低分意味着它运行了用户逻辑明确说要跳过的专家。

基于跟踪，提供你的评估。
"""
 return judge_llm.invoke(prompt)

console.print("--- [bold]评估顺序代理的过程[/bold] ---")
seq_agent_evaluation = evaluate_agent_logic(dynamic_query, final_seq_output)
console.print(seq_agent_evaluation.dict())

console.print("\n--- [bold]评估黑板系统的过程[/bold] ---")
bb_agent_evaluation = evaluate_agent_logic(dynamic_query, final_bb_output)
console.print(bb_agent_evaluation.dict())

--- 评估顺序代理的过程 ---


{'instruction_following_score': 2, 'process_efficiency_score': 3, 'justification': "代理未能遵循核心条件指令（'要么/要么'）。它没有根据新闻情绪选择一条路径，而是执行了技术和财务分析两者。这表明缺乏对用户逻辑的遵守，并导致执行不必要工作的低效过程。"}



--- 评估黑板系统的过程 ---


{'instruction_following_score': 10, 'process_efficiency_score': 10, 'justification': '代理完美遵循了用户的条件指令。在新闻分析师报告积极情绪后，系统正确选择运行技术分析师并完全跳过了财务分析师。这展示了完美的指令遵循和最优的过程效率，没有执行不必要的工作。'}


**评估输出的讨论：**
评判者的评分提供了清晰的定量判决：

- **顺序代理**将收到非常低的`instruction_following_score`（例如，2/10），因为它公然忽略了"要么/要么"条件。其`process_efficiency_score`也将很低（例如，3/10），因为它执行了明确不需要的整个分析。
- **黑板系统**将在两方面都收到接近完美的评分（例如，10/10）。评判者将识别出控制器的动态决策使系统能够精确遵循用户的指令，并通过仅激活必要的专家以最高效率运行。

这个评估提供了明确的证据，对于复杂的、涌现的问题，其中前进的路径取决于中间结果，黑板架构的灵活性远优于刚性的、预定义的工作流程。

## 总结

在这个notebook中，我们实现并修正了一个**黑板系统**，展示了其相对于顺序多代理架构的显著优势。 通过引入共享内存（黑板）和智能的、状态感知的**控制器**，我们创建了一个不仅协作，而且自适应和机会主义的系统。

正面对比显示，对于具有条件逻辑的任务，黑板系统在正确时间选择正确专家的能力导致更高效和逻辑合理的过程。 虽然它需要更复杂的控制器，但这种架构是处理刚性线性工作流程无法有效解决的那种结构不良的现实世界问题的强大工具。